Nama: : Jessica Theresia<br>
Email: : me.jessicatheresia@gmail.com<br>
ID Dicoding: : jessica_trs<br>

# **1. Import Library**

Tahap untuk mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV di google drive ke dalam variabel DataFrame.

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Memuat dataset
df = pd.read_csv("/content/drive/MyDrive/Dicoding ML Submission/Belajar Machine Learning untuk Pemula/Dataset_inisiasi.csv")

# Menampilkan 5 baris pertama
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   TransactionAmount  CustomerAge  TransactionDuration  AccountBalance  \
0              14.09           70                   81         5112.21   
1             376.24           68                  141        13758.91   
2             126.29           19                   56         1122.35   
3             184.50           26                   25         8569.06   
4              13.45           26                  198         7429.40   

  TransactionType Channel CustomerOccupation  Cluster  
0           Debit     ATM             Doctor        2  
1           Debit     ATM             Doctor        2  
2           Debit  Online            Student        1  
3           Debit  Online            Student        1  
4          Credit  Online            Student        4  


**Informasi Dataset**

untuk memastikan bahwa dataset sudah dilakukan clustering.

In [3]:
# Menampilkan informasi dasar dataset
dataset_info = df.info()
dataset_head = df.head()
missing_values = df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2399 entries, 0 to 2398
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionAmount    2399 non-null   float64
 1   CustomerAge          2399 non-null   int64  
 2   TransactionDuration  2399 non-null   int64  
 3   AccountBalance       2399 non-null   float64
 4   TransactionType      2399 non-null   object 
 5   Channel              2399 non-null   object 
 6   CustomerOccupation   2399 non-null   object 
 7   Cluster              2399 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 150.1+ KB


In [4]:
dataset_head

,TransactionAmount,CustomerAge,TransactionDuration,AccountBalance,TransactionType,Channel,CustomerOccupation,Cluster
0,14.09,70,81,5112.21,Debit,ATM,Doctor,2
1,376.24,68,141,13758.91,Debit,ATM,Doctor,2
2,126.29,19,56,1122.35,Debit,Online,Student,1
3,184.50,26,25,8569.06,Debit,Online,Student,1
4,13.45,26,198,7429.40,Credit,Online,Student,4


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set). Kemudian encode kolom kategorikal dan scalling kolom numerik.

In [5]:
# Splitting data
X = df.drop("Cluster", axis=1)
y = df["Cluster"]

# Menentukan kolom numerik & kategorikal berdasarkan X
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Encoding untuk fitur kategorikal
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# MinMax Scaling untuk fitur numerik
scaler = MinMaxScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Membagi dataset menjadi Training (80%) dan Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()

((1919, 7),
 (480, 7),
 Cluster
 2    576
 0    474
 1    432
 3    194
 4    186
 5     57
 Name: count, dtype: int64,
 Cluster
 2    144
 0    118
 1    108
 3     49
 4     47
 5     14
 Name: count, dtype: int64)

In [7]:
# SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

print("Distribusi kelas setelah SMOTE:", Counter(y_train))

Distribusi kelas setelah SMOTE: Counter({0: 576, 3: 576, 2: 576, 1: 576, 5: 576, 4: 576})


### Penjelasan Output

* (1919, 7): Training set memiliki 1.919 sampel (baris) dan 7 fitur (kolom) setelah dilakukan one-hot encoding pada fitur kategorikal.

* (480, 7): Testing set memiliki 480 sampel (baris) dan 7 fitur (kolom) dengan struktur yang sama seperti training set.

* Data telah berhasil dibagi menjadi training (80%) dan testing (20%).

* Di awal, distribusi kelas tidak seimbang, terutama kelas 5 yang memiliki sangat sedikit sampel (57 di training set dan 14 di testing set). Ketidakseimbangan ini bisa mempengaruhi performa model dalam mengenali kelas minoritas, sehingga perlu dipertimbangkan teknik seperti oversampling, undersampling, atau class weighting.

* Metode SMOTE dilakukan untuk menyeimbang distribusi sampel tiap kelas.

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

In [8]:
# 1. Membangun Model Random Forest
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced' )
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

Penjelasan

1. Random Forest
<br>
Random Forest adalah algoritma ensemble yang terdiri dari banyak Decision Tree yang dilatih secara independen menggunakan subset data dan fitur yang berbeda. Hasil prediksi diambil berdasarkan mayoritas suara (voting) untuk klasifikasi.

* Alasan Memilih Algoritma ini :

  * a. Robust terhadap Outlier dan Noise: Karena menggunakan hasil agregasi dari banyak pohon keputusan, model ini lebih tahan terhadap data yang memiliki outlier (yang sudah kita tangani sebelumnya).

  * b. Menangkap Interaksi Fitur: Dataset kita memiliki beberapa fitur numerik yang kompleks. Random Forest dapat memahami hubungan non-linear antara fitur.

  * c. Stabilitas dan Akurasi: Dengan membangun banyak pohon, model menjadi lebih stabil dan memiliki bias rendah serta varian yang moderat.

* Kaitan dengan Dataset:
  * Dataset hasil clustering mengandung variabel numerik yang memiliki distribusi yang bervariasi. Random Forest efektif dalam menangkap pola kompleks dari fitur-fitur ini.* Cocok untuk dataset dengan jumlah fitur yang tidak terlalu besar dan sudah bersih setelah penanganan outlier dan fitur redundant.

In [9]:
# 2. Membangun Model Logistic Regression
lr_model = LogisticRegression(max_iter=100, random_state=42, class_weight='balanced')
lr_model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=42)

Penjelasan

2. Logistic Regression
<br>
Logistic Regression adalah metode klasifikasi yang menggunakan fungsi sigmoid untuk memodelkan probabilitas hasil biner atau multiklas.

* Alasan Memilih Logistic Regression?

  * Sederhana dan Efisien: Algoritma ini mudah diinterpretasikan dan efisien dalam komputasi meskipun dataset memiliki banyak sampel.

  * Hubungan Linear: Jika hubungan antara fitur dan target bersifat linier, Logistic Regression akan memberikan hasil yang baik.

  * Cocok untuk Dataset Terstruktur: Logistic Regression sangat baik dalam memproses dataset yang telah dibersihkan dari outlier dan redundan.

* Kaitan dengan Dataset:

  * Dataset ini memiliki label biner hasil dari proses clustering (Cluster_KMeans: 0 atau 1), yang cocok dengan fungsi sigmoid Logistic Regression.
  * Dengan fitur yang telah ditangani outlier-nya, Logistic Regression bisa memberikan hasil akurat tanpa bias dari data ekstrem.

## **b. Evaluasi Model Klasifikasi**

**Tahapan Evaluasi Model Klasifikasi yang Dibuat untuk Melakukan Prediksi**

1. Menggunakan model Random Forest dan Logistic Regression untuk memprediksi label pada data uji (X_test),

2. Menghitung Metrik Evaluasi
  * Accuracy: Persentase prediksi yang benar dari total data uji.
  * F1-Score: Rata-rata harmonik dari Precision
  * Recall untuk mengukur keseimbangan prediksi.

3. Membuat Confusion Matrix

  * Menampilkan jumlah True Positive (TP), True Negative (TN), False Positive (FP), dan False Negative (FN) untuk kedua model.

4. Laporan Klasifikasi (Classification Report)

  * Menyediakan metrik lengkap: Precision, Recall, dan F1-Score untuk masing-masing kelas.

5. Membandingkan Performa Model

  * Menyusun hasil evaluasi dalam bentuk tabel untuk membandingkan performa Random Forest dan Logistic Regression.

In [10]:
# Melakukan prediksi menggunakan data uji
rf_preds = rf_model.predict(X_test)
lr_preds = lr_model.predict(X_test)

# Metrik evaluasi (Accuracy, F1-Score dan Confusion matrix)
rf_accuracy = accuracy_score(y_test, rf_preds)
rf_f1 = f1_score(y_test, rf_preds, average='weighted')

lr_accuracy = accuracy_score(y_test, lr_preds)
lr_f1 = f1_score(y_test, lr_preds, average='weighted')

rf_conf_matrix = confusion_matrix(y_test, rf_preds)
lr_conf_matrix = confusion_matrix(y_test, lr_preds)

rf_report = classification_report(y_test, rf_preds)
lr_report = classification_report(y_test, lr_preds)

In [11]:
# Membuat DataFrame untuk hasil evaluasi
results = pd.DataFrame({
    "Model": ["Random Forest", "Logistic Regression"],
    "Accuracy": [rf_accuracy, lr_accuracy],
    "F1-Score": [rf_f1, lr_f1]
})

In [12]:
# Menampilkan hasil evaluasi dalam tabel
print("=== Hasil Evaluasi Model ===")
print(results.to_string(index=False))

=== Hasil Evaluasi Model ===
              Model  Accuracy  F1-Score
      Random Forest       1.0       1.0
Logistic Regression       1.0       1.0


In [13]:
# Perbandingan confusion matrix
print("\n=== Confusion Matrix - Random Forest ===")
print(rf_conf_matrix)

print("\n=== Confusion Matrix - Logistic Regression ===")
print(lr_conf_matrix)


=== Confusion Matrix - Random Forest ===
[[118   0   0   0   0   0]
 [  0 108   0   0   0   0]
 [  0   0 144   0   0   0]
 [  0   0   0  49   0   0]
 [  0   0   0   0  47   0]
 [  0   0   0   0   0  14]]

=== Confusion Matrix - Logistic Regression ===
[[118   0   0   0   0   0]
 [  0 108   0   0   0   0]
 [  0   0 144   0   0   0]
 [  0   0   0  49   0   0]
 [  0   0   0   0  47   0]
 [  0   0   0   0   0  14]]


In [14]:
# Perbandingan Classification Report
print("\n=== Classification Report - Random Forest ===")
print(rf_report)

print("\n=== Classification Report - Logistic Regression ===")
print(lr_report)


=== Classification Report - Random Forest ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       118
           1       1.00      1.00      1.00       108
           2       1.00      1.00      1.00       144
           3       1.00      1.00      1.00        49
           4       1.00      1.00      1.00        47
           5       1.00      1.00      1.00        14

    accuracy                           1.00       480
   macro avg       1.00      1.00      1.00       480
weighted avg       1.00      1.00      1.00       480


=== Classification Report - Logistic Regression ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       118
           1       1.00      1.00      1.00       108
           2       1.00      1.00      1.00       144
           3       1.00      1.00      1.00        49
           4       1.00      1.00      1.00        47
           5       1.00      1.0

Penjelasan :<br>

1. Confusion Matrix
    * Kedua model menghasilkan diagonal utama (dari kiri atas ke kanan bawah) memiliki nilai selain nol, yang berarti semua prediksi benar.Tidak ada angka di luar diagonal utama, sehingga tidak ada kesalahan klasifikasi.

2. Classification Report
    * Semua nilai 1.00 (100%), artinya setiap kelas diklasifikasikan dengan sempurna tanpa kesalahan.

3. Analisis Model<br>
  * Kedua model memiliki akurasi dan F1-score 100%, yang berarti model memprediksi semua data uji dengan benar.

<br>
**Kesimpulan** <br>
1. Akurasi 100% kemungkinan besar menunjukkan overfitting, terutama karena Logistic Regression biasanya tidak mencapai performa sempurna di dataset nyata.Bisa jadi data uji memiliki pola yang sangat mirip dengan data latih, sehingga model dapat menghafalnya.<br>
2. Performa ini tidak realistis dalam kasus dunia nyata, karena model harus diuji dengan data baru yang belum pernah dilihat sebelumnya.<br>
3. Untuk memastikan model tidak overfitting, tahap selanjutnya dilakukan:
  - Pengurangan fitur atau regularisasi pada Logistic Regression.
  - Penyesuaian hyperparameter pada Random Forest (misalnya, mengurangi kedalaman pohon).

## **c. Tuning Model Klasifikasi (Optional)**

Tuning dilakukan untuk mencari kombinasi parameter terbaik yang dapat meningkatkan performa model klasifikasi yang masih bisa ditingkatkan dengan mengimplementasikan parameter, yang mana pada model ini dibuat dengan menambah kedalaman dan regulasi yang dapat mengurangi kemungkinan overfitting. Metode yang digunakan adalah GridSearchCV, yang mencoba berbagai kombinasi parameter dengan validasi silang (cross-validation) untuk menghindari overfitting dan mendapatkan model optimal.

Random Forest

In [15]:
# Hyperparameter tuning Random Forest
rf_params = {
    "n_estimators": [50, 100, 200],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5, 10],
}

# GridSearchCV
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=10, scoring="f1_weighted", n_jobs=-1)
rf_grid.fit(X_train, y_train)

best_rf_model = rf_grid.best_estimator_

Logistic Regression

In [16]:
# Hyperparameter tuning untuk Logistic Regression
lr_params = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear", "saga"],
    "max_iter": [100, 200, 500],
}

# GridSearchCV
lr_grid = GridSearchCV(LogisticRegression(max_iter=200, random_state=42), lr_params, cv=5, scoring="f1_weighted", n_jobs=-1)
lr_grid.fit(X_train, y_train)

best_lr_model = lr_grid.best_estimator_

## **d. Evaluasi Model Klasifikasi setelah Tuning**

###Logistic Regression

In [17]:
# Prediksi menggunakan model Logistic Regression setelah tuning
lr_tuned_preds = best_lr_model.predict(X_test)

# Evaluasi akurasi dan F1-Score, dan confusion matrix
lr_tuned_accuracy = accuracy_score(y_test, lr_tuned_preds)
lr_tuned_f1 = f1_score(y_test, lr_tuned_preds, average='weighted')

lr_conf_matrix_before = confusion_matrix(y_test, lr_preds)
lr_conf_matrix_after = confusion_matrix(y_test, lr_tuned_preds)

lr_tuned_report = classification_report(y_test, lr_tuned_preds)

In [18]:
# Menampilkan hasil evaluasi setelah tuning
print("\n=== Hasil Tuning Logistic Regression ===")
print(f"Best Params: {lr_grid.best_params_}")
print(f"Testing Accuracy : {lr_tuned_accuracy:.4f}")
print(f"Testing F1-Score : {lr_tuned_f1:.4f}")

# Menampilkan Confusion Matrix
print("\nConfusion Matrix Sebelum Tuning Logistic Regression:")
print(lr_conf_matrix_before)

print("\nConfusion Matrix Setelah Tuning Logistic Regression:")
print(lr_conf_matrix_after)


=== Hasil Tuning Logistic Regression ===
Best Params: {'C': 0.01, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}
Testing Accuracy : 1.0000
Testing F1-Score : 1.0000

Confusion Matrix Sebelum Tuning Logistic Regression:
[[118   0   0   0   0   0]
 [  0 108   0   0   0   0]
 [  0   0 144   0   0   0]
 [  0   0   0  49   0   0]
 [  0   0   0   0  47   0]
 [  0   0   0   0   0  14]]

Confusion Matrix Setelah Tuning Logistic Regression:
[[118   0   0   0   0   0]
 [  0 108   0   0   0   0]
 [  0   0 144   0   0   0]
 [  0   0   0  49   0   0]
 [  0   0   0   0  47   0]
 [  0   0   0   0   0  14]]


###Random Forest

In [19]:
# Prediksi menggunakan model Random Forest setelah tuning
rf_tuned_preds = best_rf_model.predict(X_test)

# Evaluasi akurasi, F1-Score dan confusion matrix
rf_tuned_accuracy = accuracy_score(y_test, rf_tuned_preds)
rf_tuned_f1 = f1_score(y_test, rf_tuned_preds, average='weighted')

rf_conf_matrix_before = confusion_matrix(y_test, rf_preds)
rf_conf_matrix_after = confusion_matrix(y_test, rf_tuned_preds)

rf_tuned_report = classification_report(y_test, rf_tuned_preds)

In [20]:
# Menampilkan hasil evaluasi setelah tuning
print("\n=== Hasil Tuning Random Forest ===")
print(f"Best Params: {rf_grid.best_params_}")
print(f"Testing Accuracy : {rf_tuned_accuracy:.4f}")
print(f"Testing F1-Score : {rf_tuned_f1:.4f}")

# Menampilkan Confusion Matrix
print("\nConfusion Matrix Sebelum Tuning Random Forest:")
print(rf_conf_matrix_before)

print("\nConfusion Matrix Setelah Tuning Random Forest:")
print(rf_conf_matrix_after)


=== Hasil Tuning Random Forest ===
Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}
Testing Accuracy : 1.0000
Testing F1-Score : 1.0000

Confusion Matrix Sebelum Tuning Random Forest:
[[118   0   0   0   0   0]
 [  0 108   0   0   0   0]
 [  0   0 144   0   0   0]
 [  0   0   0  49   0   0]
 [  0   0   0   0  47   0]
 [  0   0   0   0   0  14]]

Confusion Matrix Setelah Tuning Random Forest:
[[118   0   0   0   0   0]
 [  0 108   0   0   0   0]
 [  0   0 144   0   0   0]
 [  0   0   0  49   0   0]
 [  0   0   0   0  47   0]
 [  0   0   0   0   0  14]]


Penjelasan
* Tidak ada perubahan performa setelah tuning, menunjukkan bahwa model mungkin mengalami overfitting pada dataset ini.
* Akurasi 100% menunjukkan bahwa model mampu mengenali pola dalam data dengan sangat baik.
* Kemungkinan penyebab overfitting:
  - Data terlalu mudah dipisahkan, sehingga model tidak kesulitan melakukan klasifikasi.
  - Fitur sangat relevan dengan label, membuat model bisa langsung mengenali pola.
  - Distribusi data latih dan uji sangat mirip, sehingga model tidak benar-benar diuji dengan variasi data yang cukup.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

### Logistic Regression

Perbandingan hasil evaluasi sebelum dan sesudah tuning untuk logistic regression

In [21]:
# 1. Evaluasi sebelum tuning
lr_accuracy_before = lr_accuracy
lr_f1_before = lr_f1
lr_report_before = classification_report(y_test, lr_preds, output_dict=True)

# 2. Evaluasi setelah tuning
lr_accuracy_after = accuracy_score(y_test, lr_tuned_preds)
lr_f1_after = f1_score(y_test, lr_tuned_preds, average='weighted')
lr_report_after = classification_report(y_test, lr_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
acc_improvement = (lr_accuracy_after - lr_accuracy_before) * 100
f1_improvement = (lr_f1_after - lr_f1_before) * 100

In [22]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Logistic Regression ===")
print(f"Accuracy Sebelum Tuning : {lr_accuracy_before:.4f} ({lr_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {lr_accuracy_after:.4f} ({lr_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {lr_f1_before:.4f} ({lr_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {lr_f1_after:.4f} ({lr_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model ===")
for label in ['0', '1','2','3','4','5']:
    precision_before = lr_report_before[label]['precision']
    recall_before = lr_report_before[label]['recall']
    precision_after = lr_report_after[label]['precision']
    recall_after = lr_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan ===")
if acc_improvement < 1.0 and f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data atau analisis kembali model.")
if any(lr_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Coba data augmentation atau algoritma lain.")
else :
  print("Model sudah sangat baik. Bisa digunakan untuk deployment.")
# 7. Menampilkan hasil dalam bentuk DataFrame
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [lr_accuracy_before, lr_f1_before],
    'After Tuning': [lr_accuracy_after, lr_f1_after],
    'Improvement (%)': [acc_improvement, f1_improvement]
})

print("\n=== Ringkasan Hasil Evaluasi ===")
print(comparison_df)

=== Perbandingan Performa Logistic Regression ===
Accuracy Sebelum Tuning : 1.0000 (100.00%)
Accuracy Setelah Tuning : 1.0000 (100.00%)
Peningkatan Accuracy    : 0.00%

F1-Score Sebelum Tuning : 1.0000 (100.00%)
F1-Score Setelah Tuning : 1.0000 (100.00%)
Peningkatan F1-Score    : 0.00%


=== Analisis Kelemahan Model ===
Kelas 0:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 1:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 2:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 3:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 4:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 5:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000

=== Rekomendasi Tindakan Lanjutan ===
Peningkatan performa 

Cek Overfitting atau Underfitting

In [23]:
# Evaluasi Logistic Regression di TRAINING SET
lr_train_preds = best_lr_model.predict(X_train)
lr_train_accuracy = accuracy_score(y_train, lr_train_preds)
lr_train_f1 = f1_score(y_train, lr_train_preds, average='weighted')

# Evaluasi Logistic Regression di TESTING SET
lr_test_preds = best_lr_model.predict(X_test)
lr_test_accuracy = accuracy_score(y_test, lr_test_preds)
lr_test_f1 = f1_score(y_test, lr_test_preds, average='weighted')

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===")
print(f"Training Accuracy : {lr_train_accuracy:.4f}")
print(f"Training F1-Score : {lr_train_f1:.4f}")
print(f"Testing Accuracy  : {lr_test_accuracy:.4f}")
print(f"Testing F1-Score  : {lr_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = lr_train_accuracy - lr_test_accuracy
diff_f1 = lr_train_f1 - lr_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

# Analisis Overfitting atau Underfitting
if diff_accuracy > 0.03 or diff_f1 > 0.03:
    print("\nModel kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.")
elif lr_test_accuracy < (lr_train_accuracy * 0.85) or lr_test_f1 < (lr_train_f1 * 0.85):
    print("\nModel kemungkinan mengalami UNDERFITTING. Performa di testing set masih kurang baik.")
else:
    print("\nModel memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.")

# Cek Distribusi Prediksi
print("\n=== Distribusi Prediksi ===")
train_pred_counts = pd.Series(lr_train_preds).value_counts(normalize=True) * 100
test_pred_counts = pd.Series(lr_test_preds).value_counts(normalize=True) * 100

print("Distribusi Prediksi di Training Set:")
print(train_pred_counts)
print("\nDistribusi Prediksi di Testing Set:")
print(test_pred_counts)

=== Evaluasi Overfitting & Underfitting Model Logistic Regression ===
Training Accuracy : 1.0000
Training F1-Score : 1.0000
Testing Accuracy  : 1.0000
Testing F1-Score  : 1.0000

Perbedaan Accuracy : 0.0000
Perbedaan F1-Score : 0.0000

Model memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.

=== Distribusi Prediksi ===
Distribusi Prediksi di Training Set:
0    16.666667
3    16.666667
2    16.666667
1    16.666667
5    16.666667
4    16.666667
Name: proportion, dtype: float64

Distribusi Prediksi di Testing Set:
2    30.000000
0    24.583333
1    22.500000
3    10.208333
4     9.791667
5     2.916667
Name: proportion, dtype: float64


### Random Forest

Perbandingan hasil evaluasi sebelum dan sesudah tuning untuk Random Forest

In [24]:
# 1. Evaluasi sebelum tuning
rf_accuracy_before = rf_accuracy
rf_f1_before = rf_f1
rf_report_before = classification_report(y_test, rf_preds, output_dict=True)

# 2. Evaluasi setelah tuning
rf_accuracy_after = accuracy_score(y_test, rf_tuned_preds)
rf_f1_after = f1_score(y_test, rf_tuned_preds, average='weighted')
rf_report_after = classification_report(y_test, rf_tuned_preds, output_dict=True)

# 3. Menghitung peningkatan performa
rf_acc_improvement = (rf_accuracy_after - rf_accuracy_before) * 100
rf_f1_improvement = (rf_f1_after - rf_f1_before) * 100

In [25]:
# 4. Menampilkan hasil perbandingan sebelum & sesudah tuning
print("=== Perbandingan Performa Random Forest ===")
print(f"Accuracy Sebelum Tuning : {rf_accuracy_before:.4f} ({rf_accuracy_before * 100:.2f}%)")
print(f"Accuracy Setelah Tuning : {rf_accuracy_after:.4f} ({rf_accuracy_after * 100:.2f}%)")
print(f"Peningkatan Accuracy    : {rf_acc_improvement:.2f}%\n")

print(f"F1-Score Sebelum Tuning : {rf_f1_before:.4f} ({rf_f1_before * 100:.2f}%)")
print(f"F1-Score Setelah Tuning : {rf_f1_after:.4f} ({rf_f1_after * 100:.2f}%)")
print(f"Peningkatan F1-Score    : {rf_f1_improvement:.2f}%\n")

# 5. Identifikasi kelemahan model berdasarkan Classification Report
print("\n=== Analisis Kelemahan Model Random Forest ===")
for label in ['0', '1','2','3','4','5']:
    precision_before = rf_report_before[label]['precision']
    recall_before = rf_report_before[label]['recall']
    precision_after = rf_report_after[label]['precision']
    recall_after = rf_report_after[label]['recall']

    print(f"Kelas {label}:")
    print(f"  - Precision Sebelum: {precision_before:.4f}, Setelah: {precision_after:.4f}")
    print(f"  - Recall Sebelum   : {recall_before:.4f}, Setelah: {recall_after:.4f}")

    if precision_after < 0.8 or recall_after < 0.8:
        print(f"Precision/Recall masih di bawah 0.8 untuk kelas {label}. Bisa jadi ada ketidakseimbangan data.")

# 6. Rekomendasi tindakan lanjutan
print("\n=== Rekomendasi Tindakan Lanjutan Random Forest ===")
if rf_acc_improvement < 1.0 and rf_f1_improvement < 1.0:
    print("Peningkatan performa kecil. Pertimbangkan metode lain seperti menambah fitur atau balancing data.")
if any(rf_report_after[label]['recall'] < 0.8 for label in ['0', '1']):
    print("Recall masih rendah untuk beberapa kelas. Pertimbangkan metode lain seperti menambah fitur atau balancing data atau analisis kembali model.")
else:
    print("Model sudah sangat baik. Bisa digunakan untuk deployment.")

# 7. Menampilkan hasil dalam bentuk DataFrame
rf_comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Score'],
    'Before Tuning': [rf_accuracy_before, rf_f1_before],
    'After Tuning': [rf_accuracy_after, rf_f1_after],
    'Improvement (%)': [rf_acc_improvement, rf_f1_improvement]
})

print("\n=== Ringkasan Hasil Evaluasi Random Forest ===")
print(rf_comparison_df)

=== Perbandingan Performa Random Forest ===
Accuracy Sebelum Tuning : 1.0000 (100.00%)
Accuracy Setelah Tuning : 1.0000 (100.00%)
Peningkatan Accuracy    : 0.00%

F1-Score Sebelum Tuning : 1.0000 (100.00%)
F1-Score Setelah Tuning : 1.0000 (100.00%)
Peningkatan F1-Score    : 0.00%


=== Analisis Kelemahan Model Random Forest ===
Kelas 0:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 1:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 2:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 3:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 4:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000
Kelas 5:
  - Precision Sebelum: 1.0000, Setelah: 1.0000
  - Recall Sebelum   : 1.0000, Setelah: 1.0000

=== Rekomendasi Tindakan Lanjutan Random Forest ===

Cek Overfitting atau Underfitting

In [26]:
# Evaluasi Random Forest di TRAINING SET
rf_train_preds = rf_model.predict(X_train)
rf_train_accuracy = accuracy_score(y_train, rf_train_preds)
rf_train_f1 = f1_score(y_train, rf_train_preds, average='weighted')

# Evaluasi Random Forest di TESTING SET
rf_test_preds = rf_model.predict(X_test)
rf_test_accuracy = accuracy_score(y_test, rf_test_preds)
rf_test_f1 = f1_score(y_test, rf_test_preds, average='weighted')

# Menampilkan hasil evaluasi di kedua set
print("=== Evaluasi Overfitting & Underfitting Model Random Forest ===")
print(f"Training Accuracy : {rf_train_accuracy:.4f}")
print(f"Training F1-Score : {rf_train_f1:.4f}")
print(f"Testing Accuracy  : {rf_test_accuracy:.4f}")
print(f"Testing F1-Score  : {rf_test_f1:.4f}")

# Cek perbedaan performa
diff_accuracy = rf_train_accuracy - rf_test_accuracy
diff_f1 = rf_train_f1 - rf_test_f1
print(f"\nPerbedaan Accuracy : {diff_accuracy:.4f}")
print(f"Perbedaan F1-Score : {diff_f1:.4f}")

# Analisis Overfitting atau Underfitting
if diff_accuracy > 0.03 or diff_f1 > 0.03:
    print("\nModel kemungkinan mengalami OVERFITTING. Performa pada training jauh lebih tinggi dibanding testing.")
elif lr_test_accuracy < (lr_train_accuracy * 0.85) or lr_test_f1 < (lr_train_f1 * 0.85):
    print("\nModel kemungkinan mengalami UNDERFITTING. Performa di testing set masih kurang baik.")
else:
    print("\nModel memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.")

# Cek Distribusi Prediksi
print("\n=== Distribusi Prediksi ===")
train_pred_counts = pd.Series(rf_train_preds).value_counts(normalize=True) * 100
test_pred_counts = pd.Series(rf_test_preds).value_counts(normalize=True) * 100

print("Distribusi Prediksi di Training Set:")
print(train_pred_counts)
print("\nDistribusi Prediksi di Testing Set:")
print(test_pred_counts)

=== Evaluasi Overfitting & Underfitting Model Random Forest ===
Training Accuracy : 1.0000
Training F1-Score : 1.0000
Testing Accuracy  : 1.0000
Testing F1-Score  : 1.0000

Perbedaan Accuracy : 0.0000
Perbedaan F1-Score : 0.0000

Model memiliki generalisasi yang baik. Tidak ada indikasi overfitting atau underfitting yang signifikan.

=== Distribusi Prediksi ===
Distribusi Prediksi di Training Set:
0    16.666667
3    16.666667
2    16.666667
1    16.666667
5    16.666667
4    16.666667
Name: proportion, dtype: float64

Distribusi Prediksi di Testing Set:
2    30.000000
0    24.583333
1    22.500000
3    10.208333
4     9.791667
5     2.916667
Name: proportion, dtype: float64


Cross-Validation

In [27]:
# Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_rf_model, X_train, y_train, cv=cv, scoring='f1_weighted')
print(f"Cross-Validation F1-Score (Mean): {np.mean(cv_scores):.4f}")

# Evaluasi dengan Data Unseen
y_pred_unseen = best_rf_model.predict(X_test)
print("\n=== Classification Report pada Data Unseen ===")
print(classification_report(y_test, y_pred_unseen))

# Akurasi dan F1-Score pada Data Unseen
acc_unseen = accuracy_score(y_test, y_pred_unseen)
f1_unseen = f1_score(y_test, y_pred_unseen, average='weighted')
print(f"Testing Accuracy pada Data Unseen: {acc_unseen:.4f}")
print(f"Testing F1-Score pada Data Unseen: {f1_unseen:.4f}")

Cross-Validation F1-Score (Mean): 1.0000

=== Classification Report pada Data Unseen ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       118
           1       1.00      1.00      1.00       108
           2       1.00      1.00      1.00       144
           3       1.00      1.00      1.00        49
           4       1.00      1.00      1.00        47
           5       1.00      1.00      1.00        14

    accuracy                           1.00       480
   macro avg       1.00      1.00      1.00       480
weighted avg       1.00      1.00      1.00       480

Testing Accuracy pada Data Unseen: 1.0000
Testing F1-Score pada Data Unseen: 1.0000


**ANALISIS HASIL EVALUASI MODEL KLASIFIKASI**

1. Performa Model Setelah Tuning

  * Baik sebelum maupun setelah tuning, kedua model menunjukkan akurasi dan F1-score sebesar 100%, yang mengindikasikan bahwa model telah mampu memprediksi setiap kelas dengan sempurna pada dataset ini.
  * Cross-Validation F1-Score (Mean): 1.0000, menunjukkan bahwa model bekerja sangat baik di berbagai subset data selama validasi silang.
  * Evaluasi pada data unseen juga menghasilkan Testing Accuracy: 1.0000 dan Testing F1-Score: 1.0000, menunjukkan bahwa model memiliki kemampuan generalisasi yang sangat baik terhadap data baru.
  * Hasil tuning menunjukkan tidak ada indikasi overfitting yang signifikan pada kedua model setelah penyesuaian hyperparameter.

2. Evaluasi Overfitting

  * Training Accuracy dan Testing Accuracy sama-sama 100%, begitu pula dengan Training F1-Score dan Testing F1-Score.
  * Tidak ada selisih (0.0000) antara accuracy dan F1-score di training & testing set, yang berarti model tidak mengalami overfitting ataupun underfitting.
  * Model dapat menggeneralisasi dengan sangat baik pada data uji, setidaknya pada dataset yang digunakan.

3. Kelemahan model
  - Model menunjukkan bahwa distribusi prediksi untuk setiap kelas cukup seimbang, dengan proporsi kelas pada data latih dan data uji yang tidak jauh berbeda. Hal ini menandakan bahwa model mampu mengenali pola di setiap kelas tanpa adanya dominasi kelas tertentu, sehingga mengurangi risiko bias dalam prediksi. Namun Meskipun distribusi prediksi cukup seimbang, ada kemungkinan model lebih dominan dalam memprediksi kelas tertentu karena pola dalam dataset.
  - Akurasi dan F1-Score yang sempurna (100%) mungkin telah terlalu menyesuaikan diri dengan pola pada dataset ini dan dapat mengalami kesulitan jika diterapkan pada data yang benar-benar baru dengan distribusi yang berbeda.

4. Upaya Mengatasi Overfitting

  Dalam proses tuning model Random Forest, telah dilakukan beberapa strategi untuk mengurangi kemungkinan overfitting:

  * **Random Forest**
    * Untuk mencegah model terlalu menyesuaikan diri dengan data latih (overfitting), dilakukan beberapa langkah berikut:
    - Mengurangi jumlah estimator: Model diuji dengan 50, 100, dan 200 estimator untuk menemukan keseimbangan antara performa dan kompleksitas.
    - Membatasi kedalaman pohon keputusan: Model diuji dengan max_depth 10, 20, dan tanpa batasan (None) untuk mencegah pohon tumbuh terlalu dalam dan menangkap noise dari data.
    - Menyesuaikan minimum sampel untuk split: Parameter min_samples_split diuji dengan nilai 2, 5, dan 10 untuk memastikan setiap split memiliki cukup banyak sampel sehingga model lebih generalisasi.
    - Validasi silang (10-fold cross-validation): Digunakan untuk memastikan model bekerja secara konsisten pada berbagai subset data, meningkatkan kemampuan generalisasi.

  * **Logistic Regression**
    - Model Logistic Regression juga dituning untuk menghindari overfitting dengan cara berikut:
    - Menyesuaikan regulasi (Regularization - C Parameter): Parameter C diuji dengan nilai 0.01, 0.1, 1, 10, dan 100 untuk menemukan keseimbangan antara regularisasi yang cukup kuat tanpa mengorbankan akurasi.
    - Menentukan jenis regulasi (penalty): l1 dan l2 dibandingkan untuk melihat mana yang lebih efektif dalam menghindari overfitting.
    - Memilih solver yang optimal: liblinear dan saga diuji untuk menangani dataset yang memiliki kombinasi fitur kategorikal dan numerikal.
    - Validasi silang (5-fold cross-validation): Dilakukan untuk memastikan model memiliki performa yang konsisten di berbagai subset data.

4. Rekomendasi Penggunaan Model<br>
  
  * Performa model yang terlalu sempurna ini perlu diuji lebih lanjut menggunakan data baru yang lebih bervariasi untuk memastikan model tetap akurat.